## Utils

In [1]:
import time
import random
from pulse2percept.stimuli import Stimulus, BiphasicPulseTrain

def rand_stim(implant, n_electrodes=1):
    maxamp = 15
    maxfreq = 200
    elecs = set()
    while len(elecs) < n_electrodes:
        # add another electrode
        elecs.add(random.choice(list(implant.electrode_names)))
    stim = {}
    for elec in elecs:
        amp = random.random() * maxamp
        freq = random.random() * maxfreq
        pdur = random.expovariate(1)
        while pdur > 1000 / freq / 2 or pdur < 0.01 or pdur > 100:
            pdur = random.expovariate(1)
        stim[elec] = BiphasicPulseTrain(freq, amp, pdur, stim_dur=250)
    stimulus = Stimulus(stim)
    return stimulus


In [2]:
def time_constant(model, implant, n_elecs=1, n_samples=10, stim_func=rand_stim):
    total_time = 0
    for i in range(n_samples):
        try:
            stim = stim_func(implant, n_electrodes=n_elecs)
            implant.stim = stim
            start = time.time()
            percept = model.predict_percept(implant)
            end = time.time()
            total_time += end - start
        except Exception as e:
            print(e)
            print((freq, amp, pdur))
            continue
    print("Average prediction time (%d electrodes): %.2fms" % (n_elecs, total_time / n_samples * 1000))
    print("Average prediction time per electrode: %.2fms" % (total_time / n_samples * 1000 / n_elecs))
    return total_time / n_samples * 1000

In [3]:
def time_random(model, implant, max_elecs=30, n_samples=20, stim_func=rand_stim, plot=False):
    times = []
    elec_counts = []
    for i in range(n_samples):
        try:
            n_elecs = random.randint(1, max_elecs)
            stim = stim_func(implant, n_electrodes=n_elecs)
            implant.stim = stim
            start = time.time()
            percept = model.predict_percept(implant)
            end = time.time()
            times.append(end - start)
            elec_counts.append(n_elecs)
        except ValueError as e:
            print(e)
            print((freq, amp, pdur))
            continue
    print("Average prediction time (%d electrodes): %.2fms" % (n_elecs, sum(times) / n_samples * 1000))
    print("Average prediction time per electrode: %.2fms" % (sum(times) / sum(elec_counts) * 1000))
    
    if plot:
        n, t = zip(*sorted(zip(elec_counts, times)))
        plt.plot(n, t)
    
    return sum(times) / n_samples * 1000
    

## Testing

In [4]:
from pulse2percept.models import AxonMapModel
from pulse2percept.implants import ArgusII, DiskElectrode, ProsthesisSystem
from pulse2percept.stimuli import BiphasicPulseTrain

import numpy as np
import matplotlib.pyplot as plt

### Original (Cython, variable length axons)

#### Axon Map

In [5]:
model = AxonMapModel()
model.build()
implant = ArgusII(rot=45)

In [6]:
time_constant(model, implant, n_elecs=1, n_samples=20)
time_constant(model, implant, n_elecs=15, n_samples=2)

Average prediction time (1 electrodes): 1213.26ms


TypeError: not all arguments converted during string formatting

In [ ]:
xysteps = []
fig, ax = plt.subplots(1, 1, figsize=(10, 10))
for n_elecs in [1, 15]:
    times = []
    n_points = []
    sizes = []
    for step in xysteps:
        model = AxonMapModel(xystep=step, xrange=(), yrange=())
        model.build()
        implant = ArgusII(rot=45)
        times.append(time_constant(model, implant, n_samples=1, n_elecs=n_elecs))
        sizes.append((len(model.grid.xret), len(model.grid.yret)))
        n_points.append(len(model.grid.xret) * len(model.grid.yret))
    ax.plot(n_points, times, label=str(n_elecs))
    ax.set_xticks(n_points, [str(n_points[i]) + ":" + str(sizes[i]) for i in range(len(sizes))])
    ax.set_xlabel("Grid Size")
    ax.set_ylabel("Average Prediction Time")
    
fig.show()
    

#### Biphasic Axon Map

In [ ]:
from pulse2percept.models import BiphasicAxonMapModel

In [ ]:
model = BiphasicAxonMapModel()
model.build()
implant = ArgusII(rot=45)